In [16]:
import numpy as np

import forgi.threedee.model.coarse_grain as ftmc
import os

def get_rmsd_dict(rmsd_list):
    rmsd_dict = {}
    with open(rmsd_list, "r") as fh:
        for line in fh.readlines():
            name, rmsd = (line.rstrip()).split("\t")
            rmsd_dict[name] = float(rmsd)
    return rmsd_dict

rmsd_dict = get_rmsd_dict("../data/truncated_train_rmsd.txt")

file_path = "../data/old_training_set" #"../data/val_set"#"../data/test_set" #"../data/train_set/"

energy_file = "../data/old_train_energy.txt"#"../data/val_energy.txt"#"../data/test_energy.txt" #"../data/train_energy.txt"

write = False

#file = "../data/train_set/1c2x_C_5srRNA_C_br0.cg"
c_files = 0
c_has = 0
c_not = 0
pos_e = 0
lines = []
for file in [f for f in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, f))]:
    if file in rmsd_dict.keys():
        cg = ftmc.CoarseGrainRNA.from_bg_file(file_path + file)
        cg.radius_of_gyration
        energy = cg.infos['Energy']
    
        c_files += 1
        if energy == []:
            c_not += 1
            #print(file, energy, cg.radius_of_gyration(method="vres"), rmsd_dict[file])
        elif "ROG" in energy[0]:
            c_has +=1
            rog_en = energy[0].split(",")[0][:-4]
            if float(rog_en) > 0:
                #print(file, rog_en)
                pos_e +=1
            line = file + "\t" + rog_en + "\n"
            lines.append(line)

    

print("files", c_files)
print("ROG energy", c_has)
print("Positive energy", pos_e)
print("no energy", c_not)
print(c_has + c_not)

if write:
    with open(energy_file, "w") as fh:
        for l in lines:
            fh.write(l)


files 0
ROG energy 0
Positive energy 0
no energy 0
0


In [ ]:
import forgi.threedee.model.coarse_grain as ftmc
import forgi.threedee.utilities.vector as ftuv

file = "../data/train_set/1c2x_C_5srRNA_C_br0.cg"
cgf = ftmc.CoarseGrainRNA.from_bg_file(file)

#rotation_matrix = ftuv.rotation_matrix("x", 1.5708)
#print(rotation_matrix)
print(cgf.coords)

cgf.rotate(1.5708) #rotation_matrix)

print("\nAfter rotate\n")
print(cgf.coords)
